## **Build a financial database**
---
Conditions:
*   Database in SQLite

### 1. Libraries

In [1]:
#Import the libraries
import pandas as pd
import requests
import io
import numpy as np

### 2. Download data regarding the various stock exchanges
*   Link to csv file: https://www.iso20022.org/market-identifier-codes

In [2]:
#Using ISO 8869-1 for encoding the data
exchange_data = pd.read_csv("https://www.iso20022.org/sites/default/files/ISO10383_MIC/ISO10383_MIC.csv", encoding='iso-8859-1' )
#Extract the columns of interest
exchange_data = exchange_data[['ISO COUNTRY CODE (ISO 3166)', 'MIC', 'NAME-INSTITUTION DESCRIPTION', 'ACRONYM']]
#Rename the columns
exchange_data.rename(columns={'ISO COUNTRY CODE (ISO 3166)' : 'country_code', 
                              'MIC':'code', 
                              'NAME-INSTITUTION DESCRIPTION': 'name',
                              'ACRONYM':'acronym'}, inplace=True)
#the unique exchange id that will remain static and allow us to reference the exchanges
exchange_data['id'] = exchange_data.index                              
exchange_data.head()
#exchange_data.shape

,country_code,code,name,acronym,id
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN,0
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED,1
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE,2
3,GB,ZODM,ZODIA MARKETS,NaN,3
4,US,FNFX,FENICS FX ECN,NaN,4


### 3. Mapper for currency data

In [3]:
#mapping details for USD, GBP and Germany (EUR) 
mapper = {'US': 'USD', 'GB': 'GBP', 'DE': 'EUR'}
#country_code as a reference for mapper
exchange_data['currency'] = exchange_data['country_code'].map(mapper)
exchange_data[['id', 'name', 'currency', 'code']].head()

,id,name,currency,code
0,0,EURONEXT UK - REPORTING SERVICES,GBP,DRSP
1,1,CANADIAN SECURITIES EXCHANGE,NaN,XCNQ
2,2,CANADIAN SECURITIES EXCHANGE - PURE,NaN,PURE
3,3,ZODIA MARKETS,GBP,ZODM
4,4,FENICS FX ECN,USD,FNFX


### 4. SQL database update

In [4]:
#Import the libraries fir SQLite
import sqlite3
#Connect to database
conn = sqlite3.connect('stocks_data.db')
exchange_data[['id', 'name', 'currency', 'code']].to_sql("exchange",conn, if_exists='replace', index=False)


### 5. Scraping the tickers of the S&P500 and Dow Jones Industrial Average
*   DJ link: https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average#Components
*   SP500 link: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

In [5]:
#Scraping Wikipedia using Pandas and the read_html method - Dow Jones
dj_constituents = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average#Components')[1]
#Rename the columns to fit SQL database
dj_constituents.rename(columns={'Company':'name', 'Industry': 'industry', 'Symbol': 'ticker'}, inplace=True)
dj_constituents.head()

,name,Exchange,ticker,industry,Date added,Notes,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,As Minnesota Mining and Manufacturing,3.38%
1,American Express,NYSE,AXP,Financial services,1982-08-30,NaN,3.29%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,NaN,3.84%
3,Apple,NASDAQ,AAPL,Information technology,2015-03-19,NaN,2.76%
4,Boeing,NYSE,BA,Aerospace and defense,1987-03-12,NaN,4.01%


In [6]:
#Scraping Wikipedia using Pandas and the read_html method - S&P 500
sp_constituents = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#Rename the columns to fit SQL database
sp_constituents.rename(columns={'Symbol': 'ticker', 'Security':'name', 'Headquarters Location': 'hq_location',
                                'GICS Sector': 'sector', 'GICS Sub-Industry':'industry'},inplace=True)
sp_constituents.head()

,ticker,name,SEC filings,sector,industry,hq_location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [7]:
#Marge the two tables into one, only with relevant columns
company_table = pd.concat([sp_constituents[['name', 'industry', 'sector', 'hq_location']],dj_constituents[['name', 'industry']]])
company_table.drop_duplicates('name', inplace=True)
company_table.sort_values('name', inplace=True)
company_table.reset_index(inplace=True, drop=True)
company_table['id'] = company_table.index
company_table.head()

,name,industry,sector,hq_location,id
0,3M,Industrial Conglomerates,Industrials,"Saint Paul, Minnesota",0
1,A. O. Smith,Building Products,Industrials,"Milwaukee, Wisconsin",1
2,ADM,Agricultural Products,Consumer Staples,"Chicago, Illinois",2
3,AES,Independent Power Producers & Energy Traders,Utilities,"Arlington, Virginia",3
4,AIG,Property & Casualty Insurance,Financials,"New York City, New York",4


### 6. Security table - similar in logic to the company_table 

In [8]:
#Security_table for S&P 500
sp_security_table = sp_constituents[['ticker', 'name']].copy()
#Security_table for Dow Jones
dj_security_table = dj_constituents[['ticker', 'name']].copy()

#Marge the two tables into one, only with relevant columns
security_table = pd.concat([sp_security_table,dj_security_table])
security_table.drop_duplicates(subset='ticker',inplace=True)
security_table.sort_values('ticker', inplace=True)
security_table.reset_index(inplace=True, drop=True)
security_table['id'] = security_table.index
security_table.head()

,ticker,name,id
0,A,Agilent Technologies,0
1,AAL,American Airlines Group,1
2,AAP,Advance Auto Parts,2
3,AAPL,Apple,3
4,ABBV,AbbVie,4


### 7. Link the two tables together with their foreign key - security_table and company_table
*   correct company_id in the security table
*   security_id in the company table

In [9]:
#populating the security table with the company_id
#.to_dict() change series into label
company_id_mapper = pd.Series(company_table.id.values, index=company_table.name).to_dict()
security_table['company_id'] = security_table['name'].map(company_id_mapper)
security_table.head()

,ticker,name,id,company_id
0,A,Agilent Technologies,0,16
1,AAL,American Airlines Group,1,32
2,AAP,Advance Auto Parts,2,14
3,AAPL,Apple,3,46
4,ABBV,AbbVie,4,8


In [10]:
#populate the company table with the security_id
security_id_mapper = pd.Series(security_table.id.values, index=security_table.name).to_dict()
company_table['security_id'] = company_table['name'].map(security_id_mapper)
company_table.head()

,name,industry,sector,hq_location,id,security_id
0,3M,Industrial Conglomerates,Industrials,"Saint Paul, Minnesota",0,307.0
1,A. O. Smith,Building Products,Industrials,"Milwaukee, Wisconsin",1,39.0
2,ADM,Agricultural Products,Consumer Staples,"Chicago, Illinois",2,11.0
3,AES,Independent Power Producers & Energy Traders,Utilities,"Arlington, Virginia",3,16.0
4,AIG,Property & Casualty Insurance,Financials,"New York City, New York",4,18.0


### 8. Populate the exchange_id column and upload the data into the database
*   https://www.nasdaq.com/market-activity/stocks/screener
*   Download 2 csv files covering all the NYSE and NASDAQ traded stocks

In [11]:
nyse = pd.read_csv('nyse.csv')
nasdaq = pd.read_csv('nasdaq.csv')

#NYSLE - exchange_id = 1926
security_table['exchange_id'] = None
security_table['exchange_id'] = np.where(security_table['ticker'].isin(nyse['Symbol'].values), 1926, '')

#NASDAQ - exchange_id = 1896
security_table['exchange_id'] = np.where(security_table['ticker'].isin(nasdaq['Symbol'].values), 1896, security_table['exchange_id'])

In [12]:
#upload the data into the database
security_table.to_sql("security", conn, if_exists='replace', index=False)
company_table.to_sql("company", conn, if_exists='replace', index=False)

### 9. Download and store the price histories

In [14]:
import pandas_datareader.data as web
from tqdm import tqdm_notebook as tqdm

In [18]:
stock_pricing_dfs = []
for stock_id in tqdm(security_table['id']):
    try:
        stock_pricing_df = web.DataReader(security_table.iloc[stock_id]['ticker'],
            start='2000-01-01',
            end='2021-12-31',
            data_source='yahoo')
        
        stock_pricing_df['security_id'] = stock_id
        stock_pricing_dfs.append(stock_pricing_df)
    except:
        pass

security_price_table = pd.concat(stock_pricing_dfs)

<ipython-input-18-944aa20e3f44>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for stock_id in tqdm(security_table['id']):


  0%|          | 0/505 [00:00<?, ?it/s]

### 10. Insert the data into database table

In [19]:
#Sending the data to SQL database
security_price_table.columns = ['high', 'low', 'open', 'close', 'volume', 'adj_close', 'security_id']
security_price_table.reset_index(inplace=True)
security_price_table['id'] = security_price_table.index
security_price_table.to_sql("security_price", conn, if_exists='replace', index=False)